### Урок 2. Парсинг HTML. BeautifulSoup, MongoDB

#### Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH. Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
Наименование вакансии.<br>
Предлагаемую зарплату (отдельно минимальную и максимальную).<br>
Ссылку на саму вакансию.<br>
Сайт, откуда собрана вакансия.<br>

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
from pprint import pprint

In [2]:
def _parser_hh(vacancy):
    job_vacancies = []

    params = {
        'text': vacancy,
        'page': ''
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'
    }

    link = 'https://hh.ru/search/vacancy'

    req = requests.get(link, params=params, headers=headers)
    dom_req = bs(req.text, 'html.parser')
    last_page = int(dom_req.find_all('a', {'class': 'HH-Pager-Control'})[-2].getText())

    for page in range(0, last_page):
        params['page'] = page
        req = requests.get(link, params=params, headers=headers)
        dom_req = bs(req.text, 'html.parser')
        vacancy_items = dom_req.find_all('div', {'data-qa': 'vacancy-serp__vacancy'})
        for item in vacancy_items:
            job_vacancies.append(_parser_item_hh(item))

    return job_vacancies

In [3]:
def _parser_item_hh(item):
    job_vacancy_data = {}

    job_vacancy_link = item.find('a', {'class': 'bloko-link HH-LinkModifier'}).get('href')
    job_vacancy_name = item.find('a', {'class': 'bloko-link HH-LinkModifier'}).getText()
    job_vacancy_city = item.find('span', {'data-qa': 'vacancy-serp__vacancy-address'}).getText()
    job_vacancy_employer = item.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}).getText()
    try:
        job_vacancy_salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).getText().replace(
            u'\xa0', u'')
        job_vacancy_salary = re.split(r'\s|-', job_vacancy_salary)

        if job_vacancy_salary[0] == 'до':
            salary_min = None
            salary_max = int(job_vacancy_salary[1])

        elif job_vacancy_salary[0] == 'от':
            salary_min = int(job_vacancy_salary[1])
            salary_max = None

        else:
            salary_min = int(job_vacancy_salary[0])
            salary_max = int(job_vacancy_salary[1])

        salary_currency = job_vacancy_salary[2]

    except:
        salary_min = None
        salary_max = None
        salary_currency = None

    job_vacancy_data['name'] = job_vacancy_name 
    job_vacancy_data['city'] = job_vacancy_city
    job_vacancy_data['employer'] = job_vacancy_employer
    job_vacancy_data['salary_min'] = salary_min
    job_vacancy_data['salary_max'] = salary_max
    job_vacancy_data['salary_currency'] = salary_currency
    job_vacancy_data['link'] = job_vacancy_link
    job_vacancy_data['site'] = 'hh.ru'

    return job_vacancy_data


In [7]:

a = _parser_hh(input('Вы ищите: '))
df = pd.DataFrame(a)
b = int(input('Количество вакансий: '))
df.head(b)


Вы ищите: художник
Количество вакансий: 10


,name,city,employer,salary_min,salary_max,salary_currency,link,site
0,Помощник художника,"Москва, Щукинская",FRESQ,35000.0,65000.0,руб.,https://kostroma.hh.ru/vacancy/38983128?query=...,hh.ru
1,Стажёр-художник,"Санкт-Петербург, Ладожская",Nival,NaN,NaN,None,https://kostroma.hh.ru/vacancy/38948387?query=...,hh.ru
2,Художник-график,Москва,ООО СилаМ,30000.0,NaN,руб.,https://kostroma.hh.ru/vacancy/38923829?query=...,hh.ru
3,Художник-иллюстратор,"Москва, Электрозаводская",Мастер Снап,40000.0,50000.0,руб.,https://kostroma.hh.ru/vacancy/38967064?query=...,hh.ru
4,2D-художник,"Москва, Менделеевская",ООО ДМ2,100000.0,NaN,руб.,https://kostroma.hh.ru/vacancy/38765039?query=...,hh.ru
5,Художник по персонажам,"Санкт-Петербург, Чернышевская",SunStrike Studios,60000.0,70000.0,руб.,https://kostroma.hh.ru/vacancy/38625813?query=...,hh.ru
6,Дизайнер-художник,Москва,ООО Д-ТЕКС,60000.0,NaN,руб.,https://kostroma.hh.ru/vacancy/38698851?query=...,hh.ru
7,Дизайнер-художник,Санкт-Петербург,Пекарня ЦEХ85,NaN,NaN,None,https://kostroma.hh.ru/vacancy/38981668?query=...,hh.ru
8,"3d художник, визуализатор (стажер)",Москва,Yellow Images Inc.,50000.0,NaN,руб.,https://kostroma.hh.ru/vacancy/37709001?query=...,hh.ru
9,Дизайнер-иллюстратор,Казань,МАСТЕРСКАЯ CREED,40000.0,NaN,руб.,https://kostroma.hh.ru/vacancy/38960584?query=...,hh.ru
